In [16]:
# 필요한 라이브러리 불러오기
import pandas as pd
import folium
import json

In [2]:
# 시군별 지체 + 뇌병변 장애인수 파일 불러오기
file_path = './원본데이터/장애유형별_장애인_성_시군구별__20220623122044.csv'
num_df = pd.read_csv(file_path, encoding='euc-kr', header=1)
num_df

,시군구별,장애유형별,전체
0,서울특별시,지체,185424
1,서울특별시,뇌병변,42287
2,부산광역시,지체,82660
3,부산광역시,뇌병변,20060
4,대구광역시,지체,54677
5,대구광역시,뇌병변,13094
6,인천광역시,지체,70758
7,인천광역시,뇌병변,13068
8,광주광역시,지체,31607
9,광주광역시,뇌병변,6875


In [3]:
# 시군별로 그룹화하여 더하기
num_df_total = num_df.groupby('시군구별')[['전체']].sum()
num_df_total

,전체
시군구별,
강원도,60866
경기도,322560
경상남도,111959
경상북도,99326
광주광역시,38482
대구광역시,67771
대전광역시,41792
부산광역시,102720
서울특별시,227711


In [4]:
num_df_total.sort_values(by='전체',ascending=False)

,전체
시군구별,
경기도,322560
서울특별시,227711
경상남도,111959
부산광역시,102720
경상북도,99326
전라남도,85000
인천광역시,83826
전라북도,79790
충청남도,75633


In [5]:
# 충전기 파일 불러오기
file_path = './원본데이터/추가_전국전동휠체어급속충전기표준데이터.csv'
batt_df = pd.read_csv(file_path, encoding='euc-kr')
batt_df

,시설명,시도명,시군구명,시군구코드,소재지도로명주소,소재지지번주소,위도,경도,설치장소설명,평일운영시작시각,...,공휴일운영시작시각,공휴일운영종료시각,동시사용가능대수,공기주입가능여부,휴대전화충전가능여부,관리기관명,관리기관전화번호,데이터기준일자,제공기관코드,제공기관명
0,용산행복장애인자립생활센터,서울특별시,용산구,11170,서울특별시 용산구 원효로89길 31,NaN,37.540745,126.967358,센터 1층,09:00,...,00:00,00:00,2,Y,Y,용산행복장애인자립생활센터,02-704-0420,2021-09-15,3020000,서울특별시 용산구
1,간석3동행정복지센터,인천광역시,남동구,28200,인천광역시 남동구 용천로 186 (간석동),인천광역시 남동구 간석동 49-1,37.466111,126.714686,행정복지센터,09:00,...,00:00,00:00,2,Y,Y,인천광역시 남동구,032-453-2572,2022-03-04,3530000,인천광역시 남동구
2,간석4동행정복지센터,인천광역시,남동구,28200,인천광역시 남동구 석정로 575 (간석동),인천광역시 남동구 간석동 612-1,37.467324,126.699152,행정복지센터,09:00,...,00:00,00:00,2,Y,Y,인천광역시 남동구,032-453-2572,2022-03-04,3530000,인천광역시 남동구
3,만수2동행정복지센터,인천광역시,남동구,28200,인천광역시 남동구 만수서로 46 (만수동),인천광역시 남동구 만수동 1116-2,37.461387,126.731157,행정복지센터,09:00,...,00:00,00:00,2,Y,Y,인천광역시 남동구,032-453-2572,2022-03-04,3530000,인천광역시 남동구
4,장수서창동행정복지센터,인천광역시,남동구,28200,인천광역시 남동구 독곡로 28 (장수서창동),인천광역시 남동구 서창동 558-2,37.435880,126.747361,행정복지센터,09:00,...,00:00,00:00,2,Y,Y,인천광역시 남동구,032-453-2572,2022-03-04,3530000,인천광역시 남동구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2939,겸재정선미술관,서울특별시,강서구,11500,서울특별시 강서구 양천로47길 36,서울특별시 강서구 가양동 243-1,37.572223,126.838412,2층,10:00,...,10:00,17:00,2,Y,Y,겸재정선미술관,02-2659-2206,2022-02-14,3150000,서울특별시 강서구
2940,등촌3동주민센터,서울특별시,강서구,11500,서울특별시 강서구 화곡로63길 96,서울특별시 강서구 등촌동 705-4,37.558888,126.847919,1층,09:00,...,00:00,00:00,2,Y,Y,서울특별시 강서구 등촌3동주민센터,02-2600-7445,2022-02-14,3150000,서울특별시 강서구
2941,가양4종합사회복지관,서울특별시,강서구,11500,서울특별시 강서구 양천로57길 37,서울특별시 강서구 가양동 1478 가양4단지아파트,37.565932,126.852069,1층,09:00,...,00:00,00:00,2,Y,Y,가양4종합사회복지관,02-2668-6689,2022-02-14,3150000,서울특별시 강서구
2942,가양5종합사회복지관,서울특별시,강서구,11500,서울특별시 강서구 양천로 57길 36,서울특별시 강서구 가양동 1481 가양5단지아파트,37.564627,126.853520,1층,09:00,...,00:00,00:00,2,Y,Y,가양5종합사회복지관,02-2668-4603,2022-02-14,3150000,서울특별시 강서구


In [6]:
# 시도명으로 그룹화하여 한번에 동시사용가능한 모든 대수 더하기
batt_df_total = batt_df.groupby('시도명')[['동시사용가능대수']].sum()
batt_df_total

,동시사용가능대수
시도명,
강원도,348
경기도,827
경상남도,334
경상북도,358
광주광역시,193
대구광역시,191
대전광역시,165
부산광역시,267
서울특별시,933


In [7]:
# 전동휠체어를 보유한 사람과 사용가능대수를 나눠서
# 충전소 1대당 커버해야하는 인원수 계산하여 one_batt_에 저장
one_batt_ = round((num_df_total['전체']  * 0.067) / batt_df_total['동시사용가능대수'], 1)
one=one_batt_.sort_values(ascending=False)
one

시군구별
충청북도       36.6
세종특별자치시    29.8
경기도        26.1
부산광역시      25.8
대구광역시      23.8
경상남도       22.5
충청남도       20.9
인천광역시      20.5
울산광역시      20.4
전라남도       18.8
경상북도       18.6
대전광역시      17.0
서울특별시      16.4
전라북도       15.5
광주광역시      13.4
강원도        11.7
제주특별자치도     3.9
dtype: float64

In [8]:
## 분석(정제데이터의 시도별 충전기 1대당 커버인원수(내림차순).csv  파일 이용)
# - 충청북도가 급속충전기 1대당 감당해야 하는 인원 수가 36.6명으로 가장 높음. 인구수 대비 지체,뇌병변장애인
#  비율은 3%로 충청남도와 비슷하나 그에 비해 충전기 대수는 절반 수준임.(참고:표준데이터에 청주시의 데이터는
#  포함되어 있지 않고, 관련 공공 데이터가 존재하지 않음. 2019년 13대를 설치했다는 보도자료 있으나, 이를 반영해도 
#  1대당 32.5명으로 1위임)  + https://n.news.naver.com/mnews/article/056/0011030318  (충북 장애인 콜택시 보유율
#  전국 최하위 뉴스)  -> 결론 :충청북도의 장애인 복지에 대한 인식 수준이 미흡한 것으로 판단됨
# - 충청북도 심층 분석 진행

In [54]:
k=batt_df[batt_df['시도명']=='충청북도']
k

,시설명,시도명,시군구명,시군구코드,소재지도로명주소,소재지지번주소,위도,경도,설치장소설명,평일운영시작시각,...,공휴일운영시작시각,공휴일운영종료시각,동시사용가능대수,공기주입가능여부,휴대전화충전가능여부,관리기관명,관리기관전화번호,데이터기준일자,제공기관코드,제공기관명
30,송학면행정복지센터,충청북도,제천시,43150,충청북도 제천시 송학면 시곡포전로1길 14-2,충청북도 제천시 송학면 시곡포전로1길 14-2,37.184934,128.260912,정문입구,09:00,...,00:00,00:00,1,Y,Y,충청북도 제천시,043-641-5353,2021-12-20,4400000,충청북도 제천시
31,교동행정복지센터,충청북도,제천시,43150,충청북도 제천시 장락로 43,충청북도 제천시 장락로 43,37.146265,128.221885,후문입구,09:00,...,00:00,00:00,1,Y,Y,충청북도 제천시,043-641-5353,2021-12-20,4400000,충청북도 제천시
32,의림지동행정복지센터,충청북도,제천시,43150,충청북도 제천시 의림대로42길 34,충청북도 제천시 의림대로 42길 34,37.162075,128.215119,정문입구,09:00,...,00:00,00:00,1,Y,Y,충청북도 제천시,043-641-5353,2021-12-20,4400000,충청북도 제천시
33,중앙동행정복지센터,충청북도,제천시,43150,충청북도 제천시 독순로 61,충청북도 제천시 독순로 61,37.139348,128.210214,정문입구,09:00,...,00:00,00:00,1,Y,Y,충청북도 제천시,043-641-5353,2021-12-20,4400000,충청북도 제천시
34,남현동행정복지센터,충청북도,제천시,43150,충청북도 제천시 의병대로 147,충청북도 제천시 의병대로 147,37.134785,128.217045,정문입구,09:00,...,00:00,00:00,1,Y,Y,충청북도 제천시,043-641-5353,2021-12-20,4400000,충청북도 제천시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566,장애인회관,충청북도,보은군,44200,충청북도 보은군 보은읍 뱃들로 68-38,충청북도 보은군 보은읍 이평리 105-2,36.488682,127.723499,장애인회관,09:00,...,00:00,00:00,2,N,Y,충청북도 보은군청 주민복지과,043-540-3848,2022-01-18,4420000,충청북도 보은군
1567,보은군노인장애인복지관,충청북도,보은군,44200,충청북도 보은군 보은읍 뱃들4길 11-10,충청북도 보은군 보은읍 이평리 107,36.488364,127.723923,보은군노인장애인복지관,09:00,...,00:00,00:00,2,N,Y,충청북도 보은군청 주민복지과,043-540-3848,2022-01-18,4420000,충청북도 보은군
1822,제천장애인종합복지관,충청북도,제천시,43150,충청북도 제천시 의림대로 242,충청북도 제천시 의림대로 242,37.149342,128.216085,정문입구,09:00,...,00:00,00:00,2,Y,Y,제천장애인종합복지관,043-652-0900,2021-12-20,4400000,충청북도 제천시
1823,제천어울림체육센터,충청북도,제천시,43150,충청북도 제천시 관전로 70,충청북도 제천시 관전로 70,37.130651,128.229931,정문입구,09:00,...,00:00,00:00,1,Y,Y,충청북도 제천시,043-641-5353,2021-12-20,4400000,충청북도 제천시


In [55]:
k=k[['시설명','시도명','시군구명','위도','경도','동시사용가능대수']]
k

,시설명,시도명,시군구명,위도,경도,동시사용가능대수
30,송학면행정복지센터,충청북도,제천시,37.184934,128.260912,1
31,교동행정복지센터,충청북도,제천시,37.146265,128.221885,1
32,의림지동행정복지센터,충청북도,제천시,37.162075,128.215119,1
33,중앙동행정복지센터,충청북도,제천시,37.139348,128.210214,1
34,남현동행정복지센터,충청북도,제천시,37.134785,128.217045,1
...,...,...,...,...,...,...
1566,장애인회관,충청북도,보은군,36.488682,127.723499,2
1567,보은군노인장애인복지관,충청북도,보은군,36.488364,127.723923,2
1822,제천장애인종합복지관,충청북도,제천시,37.149342,128.216085,2
1823,제천어울림체육센터,충청북도,제천시,37.130651,128.229931,1


In [52]:
k

,시군구별,2017
0,괴산군,141.303
1,단양군,122.007
2,보은군,122.476
3,영동군,184.853
4,옥천군,182.441
5,음성군,275.973
6,제천시,453.389
7,증평군,87.569
8,진천군,165.825
9,청주시,1504.284


In [56]:
k[k['시군구명']=='청주시']

,시설명,시도명,시군구명,위도,경도,동시사용가능대수


In [17]:
a1=k[['위도','경도']].reset_index(drop=True)
a1

,위도,경도
0,37.184934,128.260912
1,37.146265,128.221885
2,37.162075,128.215119
3,37.139348,128.210214
4,37.134785,128.217045
...,...,...
67,36.488682,127.723499
68,36.488364,127.723923
69,37.149342,128.216085
70,37.130651,128.229931


In [18]:
# 충북 장애유형별 데이터 로드
df=pd.read_csv('./원본데이터/장애유형별_장애인_충북_시군구별__20220629092602.csv',encoding='euc-kr')
df

,시군구별,장애유형별,2017
0,시군구별,장애유형별,전체
1,청주시,지체,18566
2,청주시,뇌병변,3886
3,충주시,지체,6246
4,충주시,뇌병변,1140
5,제천시,지체,5895
6,제천시,뇌병변,872
7,보은군,지체,1579
8,보은군,뇌병변,249
9,옥천군,지체,2334


In [26]:
# 계산을 위해 데이터타입 변경
df['2017']=df['2017'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 1 to 22
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구별    22 non-null     object
 1   장애유형별   22 non-null     object
 2   2017    22 non-null     int32 
dtypes: int32(1), object(2)
memory usage: 572.0+ bytes


In [ ]:
# 필요없는 행 삭제
df=df.drop(0)

In [41]:
df

,시군구별,장애유형별,2017
1,청주시,지체,18566
2,청주시,뇌병변,3886
3,충주시,지체,6246
4,충주시,뇌병변,1140
5,제천시,지체,5895
6,제천시,뇌병변,872
7,보은군,지체,1579
8,보은군,뇌병변,249
9,옥천군,지체,2334
10,옥천군,뇌병변,389


In [44]:
# 시군구별 합계 도출
df = df.groupby('시군구별')[['2017']].sum()
k=df.reset_index()

In [46]:
# 전동보장구 소지 비율(6.7%)을 곱해 군별 소지 인구 추산
k['2017']=k['2017']*0.067
k

,시군구별,2017
0,괴산군,141.303
1,단양군,122.007
2,보은군,122.476
3,영동군,184.853
4,옥천군,182.441
5,음성군,275.973
6,제천시,453.389
7,증평군,87.569
8,진천군,165.825
9,청주시,1504.284


In [50]:
# 지도그리기

# 배경 맵 그리기
m=folium.Map(location=[35.229499, 129.090613], zoom_start=15, tiles='openstreetmap',control_scale=True)

# 구 단위 경계 그리기
gfile=open('./원본데이터/취.json','r',encoding='utf-8').read()
jsondata=json.loads(gfile)
folium.GeoJson(jsondata, name='municipalities').add_to(m)

# 빈도수별로 색 채우기
folium.Choropleth(
    geo_data = jsondata,
    data = k,
    columns=["시군구별", "2017"],
    fill_color = "PuRd",
    fill_opacity = 0.7,
    line_opacity = 0.7,
    key_on = "properties.name"
).add_to(m)

# 충청북도내 급속충전기 마커 찍기
for i in range(len(a1)):     
    x=a1.iloc[i,0]
    y=a1.iloc[i,1]
    folium.CircleMarker(location=[x,y],color='black',fill_color='yellow',
                    fill_opacity=0.7,weight=1.5,radius=5).add_to(m)

m

In [ ]:
## 분석
# 1. 청주시, 진천읍에 충전기가 1대도 설치되어 있지 않으며, 영동군, 보은군, 괴산군의 경우 군 면적에 비해 지나치게
#  좁은 지역에 집중되어 있는 양상
# 2. 청주시의 경우 충북 전체 인구(160만)의 절반 이상이 거주하고 있어 전동보장구 이용에 큰 어려움이 예상됨.
#  ->2019년 청주시에 13대를 추가 설치함

In [9]:
# 정제데이터 저장
#one.to_csv('one_batt.csv')